In [4]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt


V_points = [(0, 15), (5, 0), (10, 15), (6, 3), (7, 6), (3, 6), (2, 9)]
D_points = [(40, 0), (40, 8), (40, 15), (45, 15), (50, 12), (50, 4), (45, 0)]
E_points = [(20, 15), (30, 15), (20, 8), (30, 8), (20, 0), (30, 0), (20, 3)]


def calculate_distance(point1, point2):
    """Calculate Euclidean distance between two points"""
    return sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def getNeighbors(points, point_idx, epsilon):
    neighbors = []
    for i in range(len(points)):
        if i != point_idx:  
            if calculate_distance(points[point_idx], points[i]) <= epsilon:
                neighbors.append(i)
    
    return neighbors
  
letters = {'V': V_points, 'E': E_points, 'D': D_points}
letter_colors = {'V': 'purple', 'E': 'blue', 'D': 'red'}
all_points = V_points + E_points + D_points

def dbscan(points, epsilon, min_samples):
    labels = [-1] * len(points)
    cluster_id = 0
    core_points = []
    for i in range(len(points)):
        neighbors = getNeighbors(points, i, epsilon)
        if len(neighbors) >= min_samples - 1:  
            core_points.append(i)
    
    print(f"Core points found: {len(core_points)}")
    print("Core point indices:", core_points)
    print("Core point coordinates:")
    for idx in core_points:
        print(f"  {idx}: {points[idx]}")
    print()
    
    for i in range(len(points)):
        # Skip if point is already assigned to a cluster
        if labels[i] != -1:
            continue
        
        # Check if the point is a core point
        neighbors = getNeighbors(points, i, epsilon)
        if i not in core_points:
            # Not a core point, mark as noise for now
            labels[i] = -2
            continue
        
        # Start a new cluster
        print(f"Starting new cluster {cluster_id} from point {i}: {points[i]}")
        labels[i] = cluster_id
        
        # Seed with neighbors
        seed = neighbors.copy()
        
        # Expand the cluster
        j = 0
        while j < len(seed):
            current_point = seed[j]
            
            # If noise, add to cluster
            if labels[current_point] == -2:
                labels[current_point] = cluster_id
                print(f"  Added noise point {current_point}: {points[current_point]} to cluster {cluster_id}")
            
            # If unvisited
            if labels[current_point] == -1:
                # Add to cluster
                labels[current_point] = cluster_id
                print(f"  Added unvisited point {current_point}: {points[current_point]} to cluster {cluster_id}")
                
                # If it's a core point, add its neighbors to the seed
                if current_point in core_points:
                    current_neighbors = getNeighbors(points, current_point, epsilon)
                    for neighbor in current_neighbors:
                        if neighbor not in seed:
                            seed.append(neighbor)
                            print(f"    Added neighbor {neighbor}: {points[neighbor]} to seed")
            
            j += 1
        
        # Move to the next cluster
        cluster_id += 1
        print()
    
    # Identify border points and noise
    border_points = []
    noise_points = []
    
    for i in range(len(points)):
        if labels[i] == -2:  # Potential noise
            # Check if it's a border point (neighbor of a core point)
            for core_idx in core_points:
                if calculate_distance(points[i], points[core_idx]) <= epsilon:
                    labels[i] = labels[core_idx]  # Assign to the same cluster
                    border_points.append(i)
                    print(f"Point {i}: {points[i]} is a border point of cluster {labels[core_idx]}")
                    break
            
            # If still marked as noise
            if labels[i] == -2:
                noise_points.append(i)
                print(f"Point {i}: {points[i]} is noise")
    
    print(f"\nBorder points: {len(border_points)}")
    print(f"Noise points: {len(noise_points)}")
    
    return labels

def plot_dbscan_results(points, labels):
    """Plot the results of DBSCAN clustering"""
    plt.figure(figsize=(10, 6))
    
    # Get unique clusters
    unique_labels = set(labels)
    
    # Define colors for plotting
    colors = plt.cm.jet(np.linspace(0, 1, len(unique_labels)))
    
    # Plot each cluster with a different color
    for label_idx, label in enumerate(unique_labels):
        if label == -2:  # Noise points
            color = 'black'
            marker = 'x'
            markersize = 20
            label_name = 'Noise'
        else:
            color = colors[label_idx]
            marker = 'o'
            markersize = 50
            label_name = f'Cluster {label}'
        
        # Get indices of points in this cluster
        cluster_points = [i for i, l in enumerate(labels) if l == label]
        
        # Extract coordinates
        x_coords = [points[i][0] for i in cluster_points]
        y_coords = [points[i][1] for i in cluster_points]
        
        # Plot the points
        plt.scatter(x_coords, y_coords, color=color, marker=marker, 
                   s=markersize, label=label_name)
    
    # Draw the letters for reference
    for letter, points_list in letters.items():
        # Extract x and y coordinates
        x_coords = [point[0] for point in points_list]
        y_coords = [point[1] for point in points_list]
        
        # Connect points with lines according to the shape
        if letter == 'V':
            plt.plot([0, 5, 10], [15, 0, 15], color=letter_colors[letter], linestyle='--', alpha=0.5)
        elif letter == 'D':
            plt.plot([40, 40, 45, 50, 50, 45], [0, 15, 15, 12, 4, 0], color=letter_colors[letter], linestyle='--', alpha=0.5)
        elif letter == 'E':
            plt.plot([20, 30], [15, 15], color=letter_colors[letter], linestyle='--', alpha=0.5)
            plt.plot([20, 30], [8, 8], color=letter_colors[letter], linestyle='--', alpha=0.5)
            plt.plot([20, 30], [0, 0], color=letter_colors[letter], linestyle='--', alpha=0.5)
            plt.plot([20, 20], [15, 0], color=letter_colors[letter], linestyle='--', alpha=0.5)
    
    plt.title('DBSCAN Clustering Results')
    plt.xlim(-5, 55)
    plt.ylim(-2, 17)
    plt.grid(True)
    plt.legend()
    plt.savefig('dbscan_results.png')
    plt.close()

# Main execution for DBSCAN
print("="*50)
print("DBSCAN CLUSTERING")
print("="*50)

# Calculate distances between all points to help choose epsilon
print("Distance matrix for sample points:")
for i in range(5):  # Show distances for first few points
    for j in range(i+1, 5):
        dist = calculate_distance(all_points[i], all_points[j])
        print(f"Distance between {all_points[i]} and {all_points[j]}: {dist:.4f}")

# Calculate average distance within each letter
print("\nAverage distances within letters:")
for letter, points_list in letters.items():
    total_dist = 0
    count = 0
    for i in range(len(points_list)):
        for j in range(i+1, len(points_list)):
            dist = calculate_distance(points_list[i], points_list[j])
            total_dist += dist
            count += 1
    avg_dist = total_dist / count if count > 0 else 0
    print(f"Average distance within {letter}: {avg_dist:.4f}")

# Calculate minimum distances between letters
print("\nMinimum distances between letters:")
letter_keys = list(letters.keys())
for i in range(len(letter_keys)):
    for j in range(i+1, len(letter_keys)):
        letter1 = letter_keys[i]
        letter2 = letter_keys[j]
        min_dist = float('inf')
        for p1 in letters[letter1]:
            for p2 in letters[letter2]:
                dist = calculate_distance(p1, p2)
                min_dist = min(min_dist, dist)
        print(f"Minimum distance between {letter1} and {letter2}: {min_dist:.4f}")

print("\nBased on the distance analysis, we'll choose epsilon = 7.0 and min_samples = 2")
print("This should separate the letters while allowing for the shape of each letter.")
print()

# Run DBSCAN with chosen parameters
epsilon = 7.0
min_samples = 2

labels = dbscan(all_points, epsilon, min_samples)

# Print results
unique_labels = set(labels)
cluster_counts = {}
for label in unique_labels:
    count = labels.count(label)
    cluster_counts[label] = count
    if label == -2:
        print(f"Noise points: {count}")
    else:
        print(f"Cluster {label}: {count} points")

# Analyze results by letter
print("\nPoints distribution by letter and cluster:")
for letter, points_list in letters.items():
    print(f"\n{letter} points:")
    cluster_distribution = {}
    
    for i, point in enumerate(points_list):
        idx = all_points.index(point)
        cluster = labels[idx]
        
        if cluster in cluster_distribution:
            cluster_distribution[cluster].append(point)
        else:
            cluster_distribution[cluster] = [point]
    
    for cluster, points in cluster_distribution.items():
        if cluster == -2:
            print(f"  Noise: {len(points)} points")
        else:
            print(f"  Cluster {cluster}: {len(points)} points")
        for point in points:
            print(f"    {point}")

# Plot the DBSCAN results
plot_dbscan_results(all_points, labels)

# Parameter sensitivity analysis
print("\n" + "="*50)
print("PARAMETER SENSITIVITY ANALYSIS")
print("="*50)

# Try different epsilon values
epsilon_values = [5.0, 7.0, 9.0]
min_samples_values = [2, 3, 4]

for eps in epsilon_values:
    for min_samp in min_samples_values:
        print(f"\nTesting DBSCAN with epsilon = {eps} and min_samples = {min_samp}")
        test_labels = dbscan(all_points, eps, min_samp)
        
        unique_test_labels = set(test_labels)
        noise_count = test_labels.count(-2)
        cluster_count = len(unique_test_labels) - (1 if -2 in unique_test_labels else 0)
        
        print(f"Number of clusters: {cluster_count}")
        print(f"Noise points: {noise_count}")
        
        # Plot these results too
        plt.figure(figsize=(10, 6))
        colors = plt.cm.jet(np.linspace(0, 1, len(unique_test_labels)))
        
        for label_idx, label in enumerate(unique_test_labels):
            if label == -2:
                color = 'black'
                marker = 'x'
                markersize = 20
            else:
                color = colors[label_idx]
                marker = 'o'
                markersize = 50
            
            cluster_points = [i for i, l in enumerate(test_labels) if l == label]
            x_coords = [all_points[i][0] for i in cluster_points]
            y_coords = [all_points[i][1] for i in cluster_points]
            
            plt.scatter(x_coords, y_coords, color=color, marker=marker, s=markersize)
        
        # Draw letter outlines
        for letter, points_list in letters.items():
            if letter == 'V':
                plt.plot([0, 5, 10], [15, 0, 15], color=letter_colors[letter], linestyle='--', alpha=0.5)
            elif letter == 'D':
                plt.plot([40, 40, 45, 50, 50, 45], [0, 15, 15, 12, 4, 0], color=letter_colors[letter], linestyle='--', alpha=0.5)
            elif letter == 'E':
                plt.plot([20, 30], [15, 15], color=letter_colors[letter], linestyle='--', alpha=0.5)
                plt.plot([20, 30], [8, 8], color=letter_colors[letter], linestyle='--', alpha=0.5)
                plt.plot([20, 30], [0, 0], color=letter_colors[letter], linestyle='--', alpha=0.5)
                plt.plot([20, 20], [15, 0], color=letter_colors[letter], linestyle='--', alpha=0.5)
        
        plt.title(f'DBSCAN: epsilon={eps}, min_samples={min_samp}')
        plt.xlim(-5, 55)
        plt.ylim(-2, 17)
        plt.grid(True)
        plt.savefig(f'dbscan_eps_{eps}_mins_{min_samp}.png')
        plt.close()

print("\nDBSCAN clustering complete!")

DBSCAN CLUSTERING
Distance matrix for sample points:
Distance between (0, 15) and (5, 0): 15.8114
Distance between (0, 15) and (10, 15): 10.0000
Distance between (0, 15) and (6, 3): 13.4164
Distance between (0, 15) and (7, 6): 11.4018
Distance between (5, 0) and (10, 15): 15.8114
Distance between (5, 0) and (6, 3): 3.1623
Distance between (5, 0) and (7, 6): 6.3246
Distance between (10, 15) and (6, 3): 12.6491
Distance between (10, 15) and (7, 6): 9.4868
Distance between (6, 3) and (7, 6): 3.1623

Average distances within letters:
Average distance within V: 8.5094
Average distance within E: 11.1106
Average distance within D: 10.6292

Minimum distances between letters:
Minimum distance between V and E: 10.0000
Minimum distance between V and D: 30.0000
Minimum distance between E and D: 10.0000

Based on the distance analysis, we'll choose epsilon = 7.0 and min_samples = 2
This should separate the letters while allowing for the shape of each letter.

Core points found: 19
Core point indice